In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch
import pyvisa as visa
from keysight_osc import key_ask_data
import tkinter as tk
from tkinter import filedialog
import csv
import time

In [2]:
# Dedault oscilloscope
sampling_time = 100#s
memory_depth = "100000000"
oscilloscope_address = 'TCPIP0::192.168.1.36::inst0::INSTR'
rm = visa.ResourceManager()
oscilloscope = rm.open_resource(oscilloscope_address)
print(oscilloscope.query('*IDN?'))
oscilloscope.chunk_size = 1024 * 1024
time.sleep(1)
oscilloscope.write(f':ACQuire:DIGitizer 1')
print(oscilloscope.query(':ACQuire:DIGitizer?'))
oscilloscope.write(f':ACQ:POINts {memory_depth}')
memory_depth1= int(float(oscilloscope.query(':ACQuire:POINts?')))
print(f'Memory depth: {memory_depth1}')
sampling_rate1 = memory_depth1/sampling_time
print(f'Sampling rate set: {sampling_rate1/1000} kHz')
oscilloscope.write(f':ACQuire:SRATe {sampling_rate1}')#sampling_rate
time.sleep(1)
oscilloscope.write(f':TIMebase:MAIN:SCALe {sampling_time/10}')
oscilloscope.write(f':TIMebase:MAIN:POSition {sampling_time/2}')
oscilloscope.write(':SINGle')

# oscilloscope.close()

sampling_rate = int(float(oscilloscope.query(':ACQuire:SRATe:ANALog?')))
print(f'Sampling rate: {sampling_rate/1000} kHz')

KEYSIGHT TECHNOLOGIES,HD304MSO,MY64280181,10.05.2024100325

1

Memory depth: 100000000
Sampling rate set: 1000.0 kHz
Sampling rate: 10000.0 kHz


In [3]:
oscilloscope.write(':TFORce')

9

In [8]:
# Parameters
split_num = 1 # also is average number
window_seg_number = 500 # number of segments in each window 2000
# sampling_rate = 1000000 # Sa/s
zohms = 50

In [9]:
# # Get rawdata1

# oscilloscope.close()
rawdata1 = key_ask_data([1], memory_depth , time_colume=False) #Squf
# rawdata1 = key_ask_data([1], memory_depth, "keysight_l1l2_2", time_colume=True) #Squ
signal1 = rawdata1[0] #Squ
split_size = len(signal1) // split_num
signal1_split = np.array_split(signal1[:split_size*split_num], split_num)
# Welch's method for signal1
Pxx_avg_seg1 = []
for i in range(split_num):
    f1, Pxx_avg_seg1_ = welch(signal1_split[i], fs=sampling_rate, window='boxcar', nperseg=len(signal1_split[i])//window_seg_number, noverlap=None, scaling='spectrum')
    """
    window can be one of the following:
        - boxcar
        - triang
        - blackman
        - hamming
        - hann
        - bartlett
        - flattop
        - parzen
        - bohman
        - blackmanharris
        - nuttall
        - barthann
        - cosine
        - exponential
        - tukey
        - taylor
        - lanczos
    When window is boxcar, window_seg_number is 1, and noverlap is None, the result is equivalent to `scipy.signal.periodogram`.
    """
    Pxx_avg_seg1.append(Pxx_avg_seg1_)
Pxx_avg1 = np.mean(Pxx_avg_seg1, axis=0)
Pxx_avg1 = 10 * np.log10(Pxx_avg1 / zohms) + 30


('ASRL1::INSTR', 'ASRL3::INSTR', 'ASRL4::INSTR', 'ASRL5::INSTR')
KEYSIGHT TECHNOLOGIES,HD304MSO,MY64280181,10.05.2024100325

BYTeorder:LSBF

voltage range : +1.60E+00

['+1', '+0', '+100000000', '+1', '+1.00000000E-07', '+0.00000000E+00', '+0', '+4.88281257E-05', '+2.99999993E-02', '+32768\n']
Getting data from Channel 1...
WORD

Data from Channel 1 Done

OSC operation Done



In [ ]:
# Get rawdata2
# if oscilloscope.query(':TRIGger:STATus?') == 'STOP\n':
rawdata2 = key_ask_data([1], memory_depth, time_colume=False) #Antisqu
signal2 = rawdata2[0] #Antisqu
signal2_split = np.array_split(signal2[:split_size*split_num], split_num)
# Welch's method for signal2
Pxx_avg_seg2 = []
for i in range(split_num):
    f2, Pxx_avg_seg2_ = welch(signal2_split[i], fs=sampling_rate, window='boxcar', nperseg=len(signal2_split[i])//window_seg_number, noverlap=None, scaling='spectrum')
    Pxx_avg_seg2.append(Pxx_avg_seg2_)
Pxx_avg2 = np.mean(Pxx_avg_seg2, axis=0)
Pxx_avg2 = 10 * np.log10(Pxx_avg2 / zohms) + 30

In [ ]:
# Get rawdata3
# if oscilloscope.query(':TRIGger:STATus?') == 'STOP\n':
rawdata3 = key_ask_data([4], memory_depth, time_colume=False) #SNL
signal3 = rawdata3[0] #SNL
signal3_split = np.array_split(signal3[:split_size*split_num], split_num)
# Welch's method for signal3
Pxx_avg_seg3 = []
for i in range(split_num):
    f3, Pxx_avg_seg3_ = welch(signal3_split[i], fs=sampling_rate, window='boxcar', nperseg=len(signal3_split[i])//window_seg_number, noverlap=None, scaling='spectrum')
    Pxx_avg_seg3.append(Pxx_avg_seg3_)
Pxx_avg3 = np.mean(Pxx_avg_seg3, axis=0)
Pxx_avg3 = 10 * np.log10(Pxx_avg3 / zohms) + 30


In [ ]:
# # Get rawdata4
# if oscilloscope.query(':TRIGger:STATus?') == 'STOP\n':
rawdata4 = key_ask_data([4], memory_depth, time_colume=False) #SNL
signal4 = rawdata4[0] #SNL
signal4_split = np.array_split(signal4[:split_size*split_num], split_num)
# Welch's method for signal4
Pxx_avg_seg4 = []
for i in range(split_num):
    f4, Pxx_avg_seg4_ = welch(signal4_split[i], fs=sampling_rate, window='boxcar', nperseg=len(signal4_split[i])//window_seg_number, noverlap=None, scaling='spectrum')
    Pxx_avg_seg4.append(Pxx_avg_seg4_)
Pxx_avg4 = np.mean(Pxx_avg_seg4, axis=0)
Pxx_avg4 = 10 * np.log10(Pxx_avg4 / zohms) + 30


In [ ]:
# Get rawdata5
# if oscilloscope.query(':TRIGger:STATus?') == 'STOP\n':
rawdata5 = key_ask_data([4], memory_depth, time_colume=False) 
signal5 = rawdata5[0] 
signal5_split = np.array_split(signal5[:split_size*split_num], split_num)
# Welch's method for signal5
Pxx_avg_seg5 = []
for i in range(split_num):
    f5, Pxx_avg_seg5_ = welch(signal5_split[i], fs=sampling_rate, window='boxcar', nperseg=len(signal5_split[i])//window_seg_number, noverlap=None, scaling='spectrum')
    Pxx_avg_seg5.append(Pxx_avg_seg5_)
Pxx_avg5 = np.mean(Pxx_avg_seg5, axis=0)
Pxx_avg5 = 10 * np.log10(Pxx_avg5 / zohms) + 30


In [ ]:
# Get rawdata6
# if oscilloscope.query(':TRIGger:STATus?') == 'STOP\n':
rawdata6 = key_ask_data([4], memory_depth, time_colume=False) 
signal6 = rawdata6[0] 
signal6_split = np.array_split(signal6[:split_size*split_num], split_num)
# Welch's method for signal6
Pxx_avg_seg6 = []
for i in range(split_num):
    f6, Pxx_avg_seg6_ = welch(signal6_split[i], fs=sampling_rate, window='boxcar', nperseg=len(signal6_split[i])//window_seg_number, noverlap=None, scaling='spectrum')
    Pxx_avg_seg6.append(Pxx_avg_seg6_)
Pxx_avg6 = np.mean(Pxx_avg_seg6, axis=0)
Pxx_avg6 = 10 * np.log10(Pxx_avg6 / zohms) + 30


In [ ]:
# Get rawdata7
# if oscilloscope.query(':TRIGger:STATus?') == 'STOP\n':
rawdata7 = key_ask_data([4], memory_depth, time_colume=False) 
signal7 = rawdata7[0] 
signal7_split = np.array_split(signal7[:split_size*split_num], split_num)
# Welch's method for signal7
Pxx_avg_seg7 = []
for i in range(split_num):
    f7, Pxx_avg_seg7_ = welch(signal7_split[i], fs=sampling_rate, window='boxcar', nperseg=len(signal7_split[i])//window_seg_number, noverlap=None, scaling='spectrum')
    Pxx_avg_seg7.append(Pxx_avg_seg7_)
Pxx_avg7 = np.mean(Pxx_avg_seg7, axis=0)
Pxx_avg7 = 10 * np.log10(Pxx_avg7 / zohms) + 30


In [ ]:
# Get rawdata8
# if oscilloscope.query(':TRIGger:STATus?') == 'STOP\n':
rawdata8 = key_ask_data([4], memory_depth, time_colume=False) 
signal8 = rawdata8[0] 
signal8_split = np.array_split(signal8[:split_size*split_num], split_num)
# Welch's method for signal8
Pxx_avg_seg8 = []
for i in range(split_num):
    f8, Pxx_avg_seg8_ = welch(signal8_split[i], fs=sampling_rate, window='boxcar', nperseg=len(signal8_split[i])//window_seg_number, noverlap=None, scaling='spectrum')
    Pxx_avg_seg8.append(Pxx_avg_seg8_)
Pxx_avg8 = np.mean(Pxx_avg_seg8, axis=0)
Pxx_avg8 = 10 * np.log10(Pxx_avg8 / zohms) + 30


In [ ]:
# Get rawdata9
# if oscilloscope.query(':TRIGger:STATus?') == 'STOP\n':
rawdata9 = key_ask_data([4], memory_depth, time_colume=False) 
signal9 = rawdata9[0] 
signal9_split = np.array_split(signal9[:split_size*split_num], split_num)
# Welch's method for signal9
Pxx_avg_seg9 = []
for i in range(split_num):
    f9, Pxx_avg_seg9_ = welch(signal9_split[i], fs=sampling_rate, window='boxcar', nperseg=len(signal9_split[i])//window_seg_number, noverlap=None, scaling='spectrum')
    Pxx_avg_seg9.append(Pxx_avg_seg9_)
Pxx_avg9 = np.mean(Pxx_avg_seg9, axis=0)
Pxx_avg9 = 10 * np.log10(Pxx_avg9 / zohms) + 30


In [ ]:
Pxx_avg2 = Pxx_avg1
# Pxx_avg2 = Pxx_avg4
# Pxx_avg4 = Pxx_avg5

In [ ]:
signal3_split = signal7_split
# Welch's method for signal5
Pxx_avg_seg3 = []
for i in range(split_num):
    f3, Pxx_avg_seg3_ = welch(signal3_split[i], fs=sampling_rate, window='boxcar', nperseg=2, noverlap=None, scaling='spectrum')
    Pxx_avg_seg3.append(Pxx_avg_seg3_)
Pxx_avg3 = np.mean(Pxx_avg_seg3, axis=0)
Pxx_avg3 = 10 * np.log10(Pxx_avg3 / zohms) + 30

signal4_split = signal8_split
Pxx_avg_seg4 = []
for i in range(split_num):
    f4, Pxx_avg_seg4_ = welch(signal4_split[i], fs=sampling_rate, window='boxcar', nperseg=2, noverlap=None, scaling='spectrum')
    Pxx_avg_seg4.append(Pxx_avg_seg4_)
Pxx_avg4 = np.mean(Pxx_avg_seg4, axis=0)
Pxx_avg4 = 10 * np.log10(Pxx_avg4 / zohms) + 30

signal5_split = signal9_split
Pxx_avg_seg5 = []
for i in range(split_num):
    f5, Pxx_avg_seg5_ = welch(signal5_split[i], fs=sampling_rate, window='boxcar', nperseg=2, noverlap=None, scaling='spectrum')
    Pxx_avg_seg5.append(Pxx_avg_seg5_)
Pxx_avg5 = np.mean(Pxx_avg_seg5, axis=0)
Pxx_avg5 = 10 * np.log10(Pxx_avg5 / zohms) + 30




In [ ]:
# Plot
print(f'Sampling rate: {sampling_rate/1000} kHz')
yrange = 60
yref = -70
plt.figure(figsize=(10, 6))
plt.plot(f1, Pxx_avg1, label='data1')
# plt.plot(f2, Pxx_avg2, label='data2')
# plt.plot(f3, Pxx_avg3, label='data3')
# plt.plot(f4, Pxx_avg4, label='data4')
# plt.plot(f5, Pxx_avg5, label='data5')
# plt.plot(f6, Pxx_avg6, label='data6')
# plt.plot(f7, Pxx_avg7, label='data7 1000')
# plt.plot(f8, Pxx_avg8, label='data8 100')
# plt.plot(f9, Pxx_avg9, label='data9 200')
plt.title('Power Spectral Estimation')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power (dBm)')
# plt.xlim(0,50000)
plt.ylim(yref-yrange/2, yref+yrange/2)
plt.legend()
plt.grid(True)
plt.show()


/频域输出保存/

In [ ]:
# dataframe = pd.DataFrame({'Frequency': f1, 'squ(dBm)': Pxx_avg1, 'antisqu(dBm)': Pxx_avg2, 'SNL(dBm)': Pxx_avg3, 'Auxiliary(dBm)': Pxx_avg4})
# dataframe = pd.DataFrame({'Frequency': f1, 'data1(dBm)': Pxx_avg1, 'data2(dBm)': Pxx_avg2, 'data3(dBm)': Pxx_avg3, 'data4(dBm)': Pxx_avg4, 'data5(dBm)': Pxx_avg5, 'data6(dBm)': Pxx_avg6, 'data7(dBm)': Pxx_avg7, 'data8(dBm)': Pxx_avg8, 'data9(dBm)': Pxx_avg9})
# dataframe = pd.DataFrame({'Frequency': f1, 'data1(dBm)': Pxx_avg1, 'data2(dBm)': Pxx_avg2, 'data3(dBm)': Pxx_avg3, 'data4(dBm)': Pxx_avg4, 'data5(dBm)': Pxx_avg5, 'data6(dBm)': Pxx_avg6, 'data7(dBm)': Pxx_avg7, 'data8(dBm)': Pxx_avg8})
dataframe = pd.DataFrame({'Frequency': f4, 'data4(dBm)': Pxx_avg4, 'Frequency': f5, 'data5(dBm)': Pxx_avg5, 'Frequency': f6, 'data6(dBm)': Pxx_avg6, 'Frequency': f7, 'data7(dBm)': Pxx_avg7, 'Frequency': f8, 'data8(dBm)': Pxx_avg8})

dataframe.to_csv("../data/11.22data_10kSas.csv", index=False, sep=',')

/ 时域输出保存/

In [ ]:
num_samples = sampling_time * sampling_rate
preamble = oscilloscope.query(':WAV:PRE?').split(',')
# num_samples = int(preamble[2])
time_per_sample = 1/sampling_rate
x_origin = float(preamble[5])
time_array = np.array([x_origin + i * time_per_sample for i in range(num_samples)])
# time_array1 = time_array
# print(sampling_rate)
# print(sampling_time)
print(time_per_sample)
# dataframe = pd.DataFrame({'Time': time_array, 'data1': signal1, 'data2': signal2, 'data3': signal3, 'data4': signal4, 'data5': signal5, 'data6': signal6, 'data7': signal7, 'data8': signal8, 'data9': signal9})
# dataframe = pd.DataFrame({'Time': time_array, 'data1': signal1, 'data2': signal2, 'data3': signal3, 'data4': signal4, 'data5': signal5, 'data6': signal6, 'data7': signal7, 'data8': signal8})
dataframe = pd.DataFrame({'Time': time_array, 'data4': signal4, 'Time': time_array, 'data5': signal5, 'Time': time_array, 'data6': signal6, 'Time': time_array, 'data7': signal7, 'Time': time_array, 'data8': signal8})

dataframe.to_csv("../data/11.22data_10kSas_timeregion.csv", index=False, sep=',')

In [ ]:
n = len(Pxx_avg1)
Pxx_avg4 = []
Pxx_avg5 = []
for i in range(n):
    num0 = Pxx_avg1[i] - Pxx_avg3[i]
    Pxx_avg4.append(num0)
    num1 = Pxx_avg2[i] - Pxx_avg3[i]
    Pxx_avg5.append(num1)
yrange = 100
yref = 60
plt.figure(figsize=(10, 6))
plt.plot(f1, Pxx_avg4, label='data1')
plt.plot(f2, Pxx_avg5, label='data2')
# plt.plot(f3, Pxx_avg3, label='data3')
plt.title('Power Spectral Estimation')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power (dBm)')
plt.xlim(0,5)
plt.ylim(yref-yrange/2, yref+yrange/2)
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
dataframe = pd.DataFrame({'Frequency': f1, 'left-double(dBm)': Pxx_avg4, 'right-double(dBm)': Pxx_avg5})
dataframe.to_csv("../data/[17]_100s_100kSas_AM1Hz_CMRR_AC_coupled.csv", index=False, sep=',')

In [ ]:
# Save data
file_name = 'test'
preamble = oscilloscope.query(':WAV:PRE?').split(',')
num_samples = int(preamble[2])
time_per_sample = float(preamble[4])
x_origin = float(preamble[5])
time_array = np.array([x_origin + i * time_per_sample for i in range(num_samples)])
final_data = [time_array]
final_data.append(signal1)
final_data.append(signal2)
final_data.append(signal3)
print('final_data generated. Choose a folder to save the data.')
if file_name:
    # choose a folder to save the data
    root = tk.Tk()
    root.withdraw()  # hide the root window
    folder_path = filedialog.askdirectory()
    # Save waveform data to a CSV file
    print('Writing CSV...')
    BUFFER_SIZE = 1000000  # Adjust the buffer size as needed
    with open(f'{folder_path}/{file_name}_.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Time'] + ['signal1'] + ['signal2'] + ['signal3'])  # Write header
        for row in zip(*final_data):  # Transpose and write
            writer.writerow(row)
    print('CSV writing Done')

In [ ]:
# Plot
header_rows = 0
data1 = pd.read_csv('../data/[1]squ&antisqu&snl&antisqu&squ_100s_segnum500.csv', skiprows=header_rows)
x = data1['Frequency']
y1 = data1['squ(dBm)']
y2 = data1['antisqu(dBm)']
y3 = data1['SNL(dBm)']
print(f'Sampling rate: {sampling_rate/1000} kHz')
yrange = 20
yref = -77.5
plt.figure(figsize=(10, 6))
plt.plot(x, y1, label='data1')
plt.plot(x, y2, label='data2')
plt.plot(x, y3, label='data3')
plt.title('Power Spectral Estimation')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power (dBm)')
plt.xlim(0,5000)
plt.ylim(yref-yrange/2, yref+yrange/2)
plt.legend()
plt.grid(True)
plt.show()
